Bilet 1

2 вопрос (20 баллов)
Задать два двумерных тензора ar1 и ar2 размерности (4, 7), 
состоящих из случайных целых чисел в пределах 
от 0 до 10. Построить двумерный тензор размерности (4, 7), 
каждый элемент которого представляет собой 
максимум из двух значений, находящихся на аналогичной позиции в тензорах ar1, ar2. 

In [1]:
import torch

ar1 = torch.randint(0, 11, (4, 7))
ar2 = torch.randint(0, 11, (4, 7))

max_tensor = torch.max(ar1, ar2)

ar1, ar2, max_tensor

(tensor([[ 8,  8,  5,  6,  0,  8,  5],
         [ 0,  2,  1,  8, 10,  3,  4],
         [ 9,  5,  2,  9,  7,  2,  6],
         [ 2, 10,  6,  7,  4,  7, 10]]),
 tensor([[ 6,  3, 10, 10,  5,  9,  6],
         [ 9,  5,  2,  6,  0,  4,  0],
         [ 7,  5,  2,  1,  8,  2,  7],
         [ 7,  9,  6,  8,  9,  6,  8]]),
 tensor([[ 8,  8, 10, 10,  5,  9,  6],
         [ 9,  5,  2,  8, 10,  4,  4],
         [ 9,  5,  2,  9,  8,  2,  7],
         [ 7, 10,  6,  8,  9,  7, 10]]))

3 вопрос (20 баллов) 
Реализовать методы forward и backward в полносвязном слое. Входные данные обязательно должны быть в 
виде батчей (то есть кол-во измерений в inputs 2). Нельзя использовать готовую реализацию обратного 
распространения из PyTorch.
Шаблон кода, который можно использовать за основу: https://pastebin.com/Ugvjf6PA

In [10]:
import torch
from typing import Optional

class Linear:
    def __init__(self, n_inputs: int, n_neurons: int):
        self.weights = torch.randn(n_inputs, n_neurons) * 0.01
        self.biases = torch.randn(n_neurons) * 0.01
        self.inputs: Optional[torch.Tensor] = None

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        assert inputs.ndim == 2
        self.inputs = inputs
        return torch.mm(inputs, self.weights) + self.biases

    def backward(self, dvalue: torch.Tensor) -> None:
        self.dweights = torch.mm(self.inputs.T, dvalue)
        self.dbiases = dvalue.sum(axis=0)

        self.dinput = torch.mm(dvalue, self.weights.T)

linear_layer = Linear(n_inputs=3, n_neurons=2)

inputs = torch.randn(4, 3)

output = linear_layer.forward(inputs)

dvalue = torch.randn(4, 2)

linear_layer.backward(dvalue)

output, linear_layer.dweights, linear_layer.dbiases, linear_layer.dinput

(tensor([[-0.0247, -0.0049],
         [-0.0150, -0.0095],
         [ 0.0056,  0.0116],
         [ 0.0169,  0.0013]]),
 tensor([[-0.2898, -0.0815],
         [-1.9656, -1.5472],
         [ 1.7130,  1.4928]]),
 tensor([0.5855, 1.1042]),
 tensor([[-0.0140, -0.0006, -0.0159],
         [ 0.0017,  0.0016, -0.0033],
         [ 0.0105,  0.0042, -0.0011],
         [ 0.0169, -0.0041,  0.0356]]))

Bilet 2

2 вопрос (20 баллов)
Используя операции над матрицами и векторами из библиотеки PyTorch реализовать полносвязный слой с 
заданными весами weights и biases. Прогнать вектор inputs через слой и вывести результат. Входные данные 
обязательно должны быть в виде батчей (то есть кол-во измерений в inputs 2). Нельзя использовать готовые 
слои из PyTorch.

In [6]:
import torch

weights = torch.tensor([[0.2, 0.3], [0.4, 0.5], [0.6, 0.7]])
biases = torch.tensor([0.1, 0.2, 0.3])

inputs = torch.tensor([[1.0, 2.0], [3.0, 4.0]])

def fully_connected_layer(inputs, weights, biases):
    return torch.matmul(inputs, weights.T) + biases

output = fully_connected_layer(inputs, weights, biases)
output

tensor([[0.9000, 1.6000, 2.3000],
        [1.9000, 3.4000, 4.9000]])

In [7]:
#альтернатива

import torch

class Linear:
    def __init__(self, n_features, n_neurons):
        self.weights = torch.randn((n_features, n_neurons))
        self.biases = torch.randn(n_neurons)
  
    def forward(self, inputs):
        return (inputs @ self.weights) + self.biases  # torch.matmul(inputs, self.weights) + self.biases

weights = torch.randn(4, 3)
weights

inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])
inputs @ weights

linear = Linear(n_features = 4, n_neurons = 3)
linear.forward(inputs)

tensor([[-7.1360,  1.1695,  4.5237],
        [ 3.6598,  3.1153, 10.7741],
        [-2.5346, -5.4452, -1.2682]])

3 вопрос (20 баллов) 
При помощи готовых слоев PyTorch реализуйте класс модели, который:
содержит слои Conv2d, Linear;
может принимать изображение с 3 каналами;
возвращает вектор из 10 чисел для каждого фото;
числа в этом векторе принимают значения от 0 до +бесконечности. 
Кол-во слоев, каналов внутри сверток и т.д. можете выбирать сами.

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os

class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=os.cpu_count())

net = CIFAR10CNN()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

dataiter = iter(trainloader)
images, labels = next(dataiter)

images = images.to(device)

outputs = net(images)
outputs.detach().cpu()

Files already downloaded and verified


tensor([[-0.0285, -0.0446,  0.0640, -0.0069,  0.0315,  0.0365, -0.0111,  0.0319,
          0.0200, -0.0381],
        [-0.0510, -0.0524,  0.0481, -0.0273,  0.0288,  0.0438,  0.0147,  0.0179,
         -0.0061, -0.0548],
        [-0.0496, -0.0536,  0.0642, -0.0191,  0.0195,  0.0371, -0.0086,  0.0428,
          0.0285, -0.0601],
        [-0.0492, -0.0537,  0.0819, -0.0438,  0.0263,  0.0718, -0.0020,  0.0072,
         -0.0165, -0.0758]])